In [ ]:
import numpy as np
import pandas as pd
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/HW4_data.zip # FILE PATH

In [ ]:
df_train = pd.read_csv("HW4_data/public/train_label.csv", index_col=0)

In [ ]:
df_val = pd.read_csv("HW4_data/public/val_label.csv", index_col=0)

In [ ]:
# file "train" with "Edema" and "non Edema"
import os
os.chdir("/content/HW4_data")
print(os.getcwd())
os.mkdir("train")
os.mkdir("valid")

os.chdir("/content/HW4_data/train")
print(os.getcwd())
os.mkdir("Edema")
os.mkdir("non Edema")

os.chdir("/content/HW4_data/valid")
print(os.getcwd())
os.mkdir("Edema")
os.mkdir("non Edema")

/content/HW4_data
/content/HW4_data/train
/content/HW4_data/valid


In [ ]:
from shutil import copy
# classify .csv 
for i in range(10000):
  from_path = "/content/HW4_data/public/train_"+str(i)+".jpg"
  if df_train['Edema'][i] == 0:
   copy(from_path, "/content/HW4_data/train/non Edema")
  elif df_train['Edema'][i] == 1:
   copy(from_path, "/content/HW4_data/train/Edema")

In [ ]:
for i in range(1000):
  from_path = "/content/HW4_data/public/val_"+str(i)+".jpg"
  if df_val['Edema'][i] == 0:
    copy(from_path, "/content/HW4_data/valid/non Edema")
  elif df_val['Edema'][i] == 1:
    copy(from_path, "/content/HW4_data/valid/Edema")

In [ ]:
# Directory with our training dandelion pictures
train_edema_dir = os.path.join('/content/HW4_data/train/Edema')

# Directory with our training grass pictures
train_non_edema_dir = os.path.join('/content/HW4_data/train/non Edema')

# Directory with our validation dandelion pictures
valid_edema_dir = os.path.join('/content/HW4_data/valid/Edema')

# Directory with our validation grass pictures
valid_non_edema_dir = os.path.join('/content/HW4_data/valid/non Edema')

In [ ]:
train_edema_names = os.listdir(train_edema_dir)
train_non_edema_names = os.listdir(train_non_edema_dir)
validation_edema_hames = os.listdir(valid_edema_dir)
validation_non_edema_names = os.listdir(valid_non_edema_dir)

In [ ]:
# print('total training edema images:', len(os.listdir(train_edema_dir)))
# print('total training non edema images:', len(os.listdir(train_non_edema_dir)))
# print('total validation edema images:', len(os.listdir(valid_edema_dir)))
# print('total validation non edema images:', len(os.listdir(valid_non_edema_dir)))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 120 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        directory= "/content/HW4_data/train" ,  # This is the source directory for training images
        target_size=(128, 128),  # All images will be resized to 128X128
        batch_size=120,
        class_mode='binary')

# Flow validation images in batches of 19 using valid_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        directory = "/content/HW4_data/valid",  # This is the source directory for validation images
        # classes = ['edema', 'non edema'],
        target_size=(128, 128),  # All images will be resized to 128x128
        batch_size=19,
        class_mode='binary',
        shuffle=False)

Found 10000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
import tensorflow as tf
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape = (128,128,3)), 
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 49152)             0         
                                                                 
 dense (Dense)               (None, 128)               6291584   
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 6,291,713
Trainable params: 6,291,713
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator,
      steps_per_epoch=8,  
      epochs=36,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

In [ ]:
model.evaluate(validation_generator)

53/53 [==============================] - 2s 31ms/step - loss: 0.4305 - accuracy: 0.7900


[0.4304942786693573, 0.7900000214576721]

In [ ]:
import csv
output_dataroot ='STUDENTID_basic_prediction.csv' 
output_datalist =  ['','Edema'] 

from keras.preprocessing import image
import matplotlib.pyplot as plt
for fn in range(1500):
 
  # predicting images
  path ="/content/HW4_data/private/test_"+str(fn)+".jpg" 
  img = image.load_img(path, target_size=(128, 128))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)

  if classes[0]<0.5:
    tmp = np.array([i,0])
    output_datalist = np.vstack((output_datalist,tmp))
  else:
    tmp = np.array([i,1])
    output_datalist = np.vstack((output_datalist,tmp))


In [ ]:
os.chdir("/content/HW4_data")
print(os.getcwd())

# Write prediction to output csv
with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    for row in output_datalist:
        writer.writerow(row)

/content/HW4_data
